In [1]:
import pandas as pd
import warnings

In [7]:
df = pd.read_csv("../wine-ratings.csv", index_col=0)
df.head()

,name,grape,region,variety,rating,notes
0,1000 Stories Bourbon Barrel Aged Batch Blue Ca...,NaN,"Mendocino, California",Red Wine,91.0,"This is a very special, limited release of 100..."
1,1000 Stories Bourbon Barrel Aged Gold Rush Red...,NaN,California,Red Wine,89.0,The California Gold Rush was a period of coura...
2,1000 Stories Bourbon Barrel Aged Gold Rush Red...,NaN,California,Red Wine,90.0,The California Gold Rush was a period of coura...
3,1000 Stories Bourbon Barrel Aged Zinfandel 2013,NaN,"North Coast, California",Red Wine,91.0,"The wine has a deep, rich purple color. An int..."
4,1000 Stories Bourbon Barrel Aged Zinfandel 2014,NaN,California,Red Wine,90.0,Batch #004 is the first release of the 2014 vi...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32780 entries, 0 to 32779
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     32780 non-null  object 
 1   grape    0 non-null      float64
 2   region   32777 non-null  object 
 3   variety  32422 non-null  object 
 4   rating   32780 non-null  float64
 5   notes    32780 non-null  object 
dtypes: float64(2), object(4)
memory usage: 1.8+ MB


In [9]:
df.drop(columns=['grape'], axis=1, inplace=True)
df.head()

,name,region,variety,rating,notes
0,1000 Stories Bourbon Barrel Aged Batch Blue Ca...,"Mendocino, California",Red Wine,91.0,"This is a very special, limited release of 100..."
1,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,Red Wine,89.0,The California Gold Rush was a period of coura...
2,1000 Stories Bourbon Barrel Aged Gold Rush Red...,California,Red Wine,90.0,The California Gold Rush was a period of coura...
3,1000 Stories Bourbon Barrel Aged Zinfandel 2013,"North Coast, California",Red Wine,91.0,"The wine has a deep, rich purple color. An int..."
4,1000 Stories Bourbon Barrel Aged Zinfandel 2014,California,Red Wine,90.0,Batch #004 is the first release of the 2014 vi...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32780 entries, 0 to 32779
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     32780 non-null  object 
 1   region   32777 non-null  object 
 2   variety  32422 non-null  object 
 3   rating   32780 non-null  float64
 4   notes    32780 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB


In [11]:
df.dropna(axis=0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32421 entries, 0 to 32779
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     32421 non-null  object 
 1   region   32421 non-null  object 
 2   variety  32421 non-null  object 
 3   rating   32421 non-null  float64
 4   notes    32421 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB


In [12]:
data = df.sample(100).to_dict('records')
print(len(data))

100


In [13]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [14]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')
qdrant = QdrantClient(":memory:")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
qdrant.recreate_collection(collection_name='top_wines',
                           vectors_config=models.VectorParams(
                               size=encoder.get_sentence_embedding_dimension(),
                               distance=models.Distance.COSINE
                           ))

/var/folders/y0/4vdn8x4s1093zn89k44s0xdr0000gn/T/ipykernel_56142/232287267.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(collection_name='top_wines',


True

In [18]:
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        )for idx,doc in enumerate(data)
    ]
)

In [21]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [22]:
hits = qdrant.search(
    collection_name="top_wines",
    query_vector = encoder.encode(user_prompt).tolist(),
    limit = 3
)

for hit in hits:
    print(hit.payload,":score:", hit.score)

{'name': 'Achaval-Ferrer Mendoza Malbec 2018', 'region': 'Mendoza, Argentina', 'variety': 'Red Wine', 'rating': 91.0, 'notes': 'Malbec Mendoza is an inevitable reflection of the variety that is emblematic to our country, the mountainous ranges of the Andes and the sun. Composed of various vineyards, located in the major geographic references that the Cuyana province offers, this young wine, potent and velvety wine, expresses pleasure, smooth and soft, combining land and flavors in one clear strong message. This wine explains Malbec as well as Mendoza in one universal language.'} :score: 0.6843591840210076
{'name': 'Gallo of Sonoma Cabernet Sauvignon 1998', 'region': 'Sonoma County, California', 'variety': 'Red Wine', 'rating': 86.0, 'notes': 'This is a full, rich wine with concentrated flavors of black cherries, currant, and spice. Excellent structure from the tannins and appealing depth from the months of oak aging.'} :score: 0.5383035844367381
{'name': 'Inglenook Rubicon (1.5 Liter M

In [23]:
search_results = [hit.payload for hit in hits]

In [24]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ChatCompletionMessage(content="[{'name': 'Achaval-Ferrer Mendoza Malbec 2018', 'region': 'Mendoza, Argentina', 'variety': 'Red Wine', 'rating': 91.0, 'notes': 'Malbec Mendoza is an inevitable reflection of the variety that is emblematic to our country, the mountainous ranges of the Andes and the sun. Composed of various vineyards, located in the major geographic references that the Cuyana province offers, this young wine, potent and velvety wine, expresses pleasure, smooth and soft, combining land and flavors in one clear strong message. This wine explains Malbec as well as Mendoza in one universal language.'}, {'name': 'Gallo of Sonoma Cabernet Sauvignon 1998', 'region': 'Sonoma County, California', 'variety': 'Red Wine', 'rating': 86.0, 'notes': 'This is a full, rich wine with concentrated flavors of black cherries, currant, and spice. Excellent structure from the tannins and appealing depth from the months of oak aging.'}, {'name': 'Inglenook Rubicon (1.5 Liter Magnum) 1995', 'regio